In [1]:
import pandas as pd
import re
import json
import xml.etree.ElementTree as xet
import requests as req
import time
import glob
import csv
import regex
import io
import tqdm
from bs4 import BeautifulSoup

In [2]:
def parse_sitemap(fn):

    xml = xet.parse(fn)
    xml_root = xml.getroot()
    news_list = []
    
    for news in xml_root:
        try:
            result = {}
            result['url'] = news.find('{http://www.sitemaps.org/schemas/sitemap/0.9}loc').text
            lastmod = news.find('{http://www.sitemaps.org/schemas/sitemap/0.9}lastmod')
            if lastmod is not None:
                result['lastmod'] = lastmod.text

            #title = news.find('{http://www.sitemaps.org/schemas/sitemap/0.9}news:title').text
            #keywords = news.find('{http://www.sitemaps.org/schemas/sitemap/0.9}news:keywords').text
            #news_list.append(dict(zip(['url', 'lastmod'], [url, lastmod])))
            news_list.append(result)
        except:
            continue

    return news_list

In [3]:
sitemaps = parse_sitemap(io.StringIO(req.get('https://ria.ru/sitemap_article_index.xml').content.decode()))
sitemaps_filtered = []
for sitemap in sitemaps:
    if re.search(r'2020|2019', sitemap['lastmod']):
        sitemaps_filtered.append(sitemap)

In [4]:
news_db = []

for fn in tqdm.tqdm_notebook(sitemaps_filtered):
    s1 = time.time()

    try:
        text = io.StringIO(req.get(fn['url']).content.decode())
        news = parse_sitemap(text)
    except Exception as e:
        print(e)
        continue
        
    news_db += news
    s2 = time.time()
    time.sleep(max(1 - s2 + s1, 0))

<ipython-input-4-73c316e576b5>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for fn in tqdm.tqdm_notebook(sitemaps_filtered):


In [5]:
df = pd.DataFrame(news_db)
df['lastmod'] = pd.to_datetime(df['lastmod'], format='%Y-%m-%dT%H:%M:%S', utc=True)
df.head()

,url,lastmod
0,https://ria.ru/20201213/vzaimoposch-1589022219...,2020-12-13 15:57:36+00:00
1,https://ria.ru/20201213/tramp-1589022094.html,2020-12-13 16:03:03+00:00
2,https://ria.ru/20201213/karabakh-1589021479.html,2020-12-13 15:45:06+00:00
3,https://ria.ru/20201213/khoroshevtsev-15890170...,2020-12-13 14:50:21+00:00
4,https://ria.ru/20201213/moskva-1589016818.html,2020-12-13 14:50:24+00:00


In [6]:
df[(df.lastmod >= '2019-11-01')].to_json('news_ria.json', date_unit='s', date_format='iso')
df[(df.lastmod >= '2019-11-01')].shape
#df[(df.lastmod >= '2018-01-01') & (df.lastmod < '2019-07-01')].shape

(254131, 2)

# Parsing

In [129]:
def parse_ria_news(url):
    soup = BeautifulSoup(req.get(url).content, features='lxml')
    news_text_list = soup.find_all(name="div", attrs={'class': "article__text"})

    if news_text_list:
        news_text = ''
        for news in news_text_list:
            news_text += " " + news.get_text(strip=True)
    else:
        news_text = ''

    tags_elem = soup.find(name="meta", attrs={'name': "analytics:tags"})
    tags = re.sub(r'[\n\r]+', ' ', tags_elem.get('content') or '').replace(u'\xa0', u' ') if tags_elem else ''
    
    title_elem = soup.find(name="meta", attrs={'name': "analytics:title"})
    title = re.sub(r'[\n\r]+', ' ', title_elem.get('content') or '').replace(u'\xa0', u' ') if title_elem else ''

    return {'news_text': news_text, 'tags': tags, 'title': title}